In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
# IMPORTING THE PREPROCESSED DATA - CONTAINING IMAGES TOO
import pandas as pd
df = pd.read_pickle('withpixel_gender.pkl')

In [3]:
"""
full path - the path of image file
gender - 0,1 denoting male female
age - the category of the person with age group
pixels - image encoded as matrix in dataframe
"""
df

,full_path,gender,age,pixels
0,[17/10000217_1981-05-05_2009.jpg],1.0,28,"[255.0, 255.0, 255.0, 255.0, 255.0, 255.0, 255..."
2,[12/100012_1948-07-03_2008.jpg],1.0,60,"[92.0, 97.0, 91.0, 89.0, 94.0, 90.0, 91.0, 96...."
4,[16/10002116_1971-05-31_2012.jpg],0.0,41,"[61.0, 30.0, 10.0, 61.0, 30.0, 10.0, 61.0, 30...."
5,[02/10002702_1960-11-09_2012.jpg],0.0,52,"[97.0, 122.0, 178.0, 97.0, 122.0, 178.0, 97.0,..."
6,[41/10003541_1937-09-27_1971.jpg],1.0,34,"[190.0, 189.0, 194.0, 204.0, 203.0, 208.0, 203..."
...,...,...,...,...
62321,[38/9996938_1937-02-15_1968.jpg],1.0,31,"[71.0, 71.0, 71.0, 71.0, 71.0, 71.0, 71.0, 71...."
62322,[46/9996946_1943-11-01_1968.jpg],1.0,25,"[54.0, 54.0, 54.0, 44.0, 44.0, 44.0, 28.0, 28...."
62323,[49/9996949_1937-04-17_1963.jpg],1.0,26,"[41.0, 41.0, 41.0, 29.0, 29.0, 29.0, 22.0, 22...."
62325,[09/9998109_1972-12-27_2013.jpg],1.0,41,"[137.0, 174.0, 94.0, 137.0, 174.0, 94.0, 137.0..."


In [4]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

In [16]:
classes = 2 # 2 for gender
target = df['gender'].values
target_classes = keras.utils.to_categorical(target, classes)

features = []

# limiting the target classes, and featues to limit memory usage
# both target and feature must match with the split dataset 
limit_dataset = 10000

target_classes = target_classes[:limit_dataset]

for i in range(0, df.shape[0]):
    features.append(df['pixels'].values[i])
    if len(features)>=limit_dataset:
        print('Done - decided number of features are collected')
        break

# convering the list into numpy - that can be used for batch training
features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

Done - decided number of features are collected


In [17]:
# managing splits in dataset for training and evaluation
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

print(len(features))
print(len(target_classes))
print(len(train_x),len(test_x))

10000
10000
7000 3000


In [18]:
# since memory consumption is very high -we need to delete the df to get it back
# del df
# del features # 20 gb ram freeedddd

In [19]:
# Creating the base VGG Face Model

from tensorflow.keras import Sequential
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation


# vgg-face model 

model = Sequential()
model.add(ZeroPadding2D((1,1), input_shape=(224,224,3)))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,(3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128,(3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7,7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1,1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1,1)))
model.add(Flatten())
model.add(Activation('softmax'))



In [20]:
# pretrained vgg weights availble on drive
# you can find it here: https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing

model.load_weights('vgg_face_weights.h5')

In [21]:
# Changing the last layers of network to get 100 predictions for classes

from tensorflow.keras import Model

# lock the layer weights for early layers 
# - they could already detect some patterns
# fitting the network from scratch might cause to lose this info
# freeze all layers except last 3 conv layers - 2622 units
# just 101 units for age prediction task
# then add custom layer for 101 layers

# to not lose the training done before in pretrained weights
for layer in model.layers[:-7]:
    layer.trainable = False

base_model_output = Sequential()
base_model_output = Convolution2D(2, (1, 1), name='predictions')(model.layers[-4].output)
base_model_output = Flatten()(base_model_output)
base_model_output = Activation('softmax')(base_model_output)
 
gender_model = Model(inputs=model.input, outputs=base_model_output)

In [22]:
gender_model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_13_input (Inp [(None, 224, 224, 3)]     0         
_________________________________________________________________
zero_padding2d_13 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 114, 114, 64)     

In [26]:
from tensorflow.keras.callbacks import ModelCheckpoint
scores = []
epochs = 250; batch_size = 256

# 101 classes so categorical cross entropy - but have to read once again
gender_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='gender_model.hdf5',monitor='val_loss',
                verbose=1, save_best_only=True, mode='auto')

for i in range(epochs):
    print("epoch ",i)
 
    ix_train = np.random.choice(train_x.shape[0], size=batch_size)
 
    score = gender_model.fit(train_x[ix_train], train_y[ix_train],
        epochs=1, validation_data=(test_x, test_y), callbacks=[checkpointer])

scores.append(score)

epoch  0


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.